# Applying Schemas to JSON Data

Apache Spark&trade; and Azure Databricks&reg; provide a number of ways to project structure onto semi-structured data allowing for quick and easy access.

-sandbox
### Schemas

Schemas are at the heart of data structures in Spark.
**A schema describes the structure of your data by naming columns and declaring the type of data in that column.** 
Rigorously enforcing schemas leads to significant performance optimizations and reliability of code.

Why is open source Spark so fast, and why is [Databricks Runtime even faster?](https://databricks.com/blog/2017/07/12/benchmarking-big-data-sql-platforms-in-the-cloud.html) While there are many reasons for these performance improvements, two key reasons are:<br><br>
* First and foremost, Spark runs first in memory rather than reading and writing to disk. 
* Second, using DataFrames allows Spark to optimize the execution of your queries because it knows what your data looks like.

Two pillars of computer science education are data structures, the organization and storage of data and algorithms, and the computational procedures on that data.  A rigorous understanding of computer science involves both of these domains. When you apply the most relavant data structures, the algorithms that carry out the computation become significantly more eloquent.

In the roadmap for ETL, this is the **Apply Schema** step:

<img src="https://files.training.databricks.com/images/eLearning/ETL-Part-1/ETL-Process-2.png" style="border: 1px solid #aaa; border-radius: 10px 10px 10px 10px; box-shadow: 5px 5px 5px #aaa"/>

### Schemas with Semi-Structured JSON Data

**Tabular data**, such as that found in CSV files or relational databases, has a formal structure where each observation, or row, of the data has a value (even if it's a NULL value) for each feature, or column, in the data set.  

**Semi-structured data** does not need to conform to a formal data model. Instead, a given feature may appear zero, once, or many times for a given observation.  

Semi-structured data storage works well with hierarchical data and with schemas that may evolve over time.  One of the most common forms of semi-structured data is JSON data, which consists of attribute-value pairs.

Run the following cell to mount the data:

In [5]:
%run "./Includes/Classroom-Setup"

Print the first few lines of a JSON file holding ZIP Code data.

In [7]:
%fs head /mnt/training/zips.json

### Schema Inference

Import data as a DataFrame and view its schema with the `printSchema()` DataFrame method.

In [9]:
zipsDF = spark.read.json("/mnt/training/zips.json")
zipsDF.printSchema()

Store the schema as an object by calling `.schema` on a DataFrame. Schemas consist of a `StructType`, which is a collection of `StructField`s.  Each `StructField` gives a name and a type for a given field in the data.

In [11]:
zipsSchema = zipsDF.schema
print(type(zipsSchema))

[field for field in zipsSchema]

### User-Defined Schemas

Spark infers schemas from the data, as detailed in the example above.  Challenges with inferred schemas include:  
<br>
* Schema inference means Spark scans all of your data, creating an extra job, which can affect performance
* Consider providing alternative data types (for example, change a `Long` to a `Integer`)
* Consider throwing out certain fields in the data, to read only the data of interest

To define schemas, build a `StructType` composed of `StructField`s.

Import the necessary types from the `types` module. Build a `StructType`, which takes a list of `StructField`s.  Each `StructField` takes three arguments: the name of the field, the type of data in it, and a `Boolean` for whether this field can be `Null`.

In [14]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

zipsSchema2 = StructType([
  StructField("city", StringType(), True), 
  StructField("pop", IntegerType(), True) 
])

-sandbox
Apply the schema using the `.schema` method. This `read` returns only  the columns specified in the schema and changes the column `pop` from `LongType` (which was inferred above) to `IntegerType`.

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> A `LongType` is an 8-byte integer ranging up to 9,223,372,036,854,775,807 while `IntegerType` is a 4-byte integer ranging up to 2,147,483,647.  Since no American city has over two billion people, `IntegerType` is sufficient.

In [16]:
zipsDF2 = (spark.read
  .schema(zipsSchema2)
  .json("/mnt/training/zips.json")
)

display(zipsDF2)

-sandbox
### Primitive and Nonprimitive Types

The Spark [`types` package](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#module-pyspark.sql.types) provides the building blocks for constructing schemas.

A primitive type contains the data itself.  The most common primitive types include:

| Numeric | General | Time |
|-----|-----|
| `FloatType` | `StringType` | `TimestampType` | 
| `IntegerType` | `BooleanType` | `DateType` | 
| `DoubleType` | `NullType` | |
| `LongType` | | |
| `ShortType` |  | |

Non-primitive types are sometimes called reference variables or composite types.  Technically, non-primitive types contain references to memory locations and not the data itself.  Nonprimitive types are the composite of a number of primitive types such as an Array of the primitive type `Integer`.

The two most common composite types are `ArrayType` and `MapType`. These types allow for a given field to contain an arbitrary number of elements in either an Array/List or Map/Dictionary form.

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> See the [Spark documentation](http://spark.apache.org/docs/latest/sql-programming-guide.html#data-types) for a complete picture of types in Spark.

The ZIP Code dataset contains an array with the latitude and longitude of the cities.  Use an `ArrayType`, which takes the primitive type of its elements as an argument.

In [19]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, ArrayType, FloatType

zipsSchema3 = StructType([
  StructField("city", StringType(), True), 
  StructField("loc", 
    ArrayType(FloatType(), True), True),
  StructField("pop", IntegerType(), True)
])

Apply the schema using the `.schema()` method and observe the results.  Expand the array values in the column `loc` to explore further.

In [21]:
zipsDF3 = (spark.read
  .schema(zipsSchema3)
  .json("/mnt/training/zips.json")
)
display(zipsDF3)

## Exercise 1: Exploring JSON Data

<a href="https://archive.ics.uci.edu/ml/datasets/UbiqLog+(smartphone+lifelogging)">Smartphone data from UCI Machine Learning Repository</a> is available under `/mnt/training/UbiqLog4UCI`. This is log data from the open source project [Ubiqlog](https://github.com/Rezar/Ubiqlog).

Import this data and define your own schema.

### Step 1: Import the Data

Import data from `/mnt/training/14_F/log*`. (This is the log files from a given user.)

Look at the head of one file from the data set.  Use `/mnt/training/UbiqLog4UCI/14_F/log_1-6-2014.txt`.

In [25]:
# ANSWER
print(dbutils.fs.head('/mnt/training/UbiqLog4UCI/14_F/log_1-6-2014.txt', 200)) # this evaluates to the thing as %fs head /mnt/training/UbiqLog4UCI/14_F/log_1-6-2014.txt

Read the data and save it to `smartphoneDF`. Read the logs using a `*` in your path like `/mnt/training/UbiqLog4UCI/14_F/log*`.

In [27]:
# ANSWER
smartphoneDF = spark.read.json("/mnt/training/UbiqLog4UCI/14_F/log*")

In [28]:
# TEST - Run this cell to test your solution
from pyspark.sql.functions import desc

cols = set(smartphoneDF.columns)
expected = {'Application', 'Bluetooth', 'Call', 'Location', 'SMS', 'WiFi', '_corrupt_record'}
sample = smartphoneDF.orderBy(desc("Application")).first()[0][0]

dbTest("ET1-P-05-01-01", 25372, smartphoneDF.count())
dbTest("ET1-P-05-01-02", expected <= cols, True)
dbTest("ET1-P-05-01-03", expected >= cols, True)
dbTest("ET1-P-05-01-04", '12-9-2013 21:30:02', sample)

print("Tests passed!")

### Step 2: Explore the Inferred Schema

Print the schema to get a sense for the data.

In [30]:
# ANSWER
smartphoneDF.printSchema()

The schema shows:  

* Six categories of tracked data 
* Nested data structures
* A field showing corrupt records

## Exercise 2: Creating a User Defined Schema

### Step 1: Set Up Your workflow

Often the hardest part of a coding challenge is setting up a workflow to get continuous feedback on what you develop.

Start with the import statements you need, including functions from two main packages:

| Package | Function |
|---------|---------|
| `pyspark.sql.types` | `StructType`, `StructField`, `StringType` |
| `pyspark.sql.functions` | `col` |

In [34]:
# ANSWER

from pyspark.sql.types import StructType, StructField, StringType
from pyspark.sql.functions import col

The **SMS** field needs to be parsed. Create a placeholder schema called `schema` that's a `StructType` with one `StructField` named **SMS** of type `StringType`. This imports the entire attribute (even though it contains nested entities) as a String.  

This is a way to get a sense for what's in the data and make a progressively more complex schema.

In [36]:
# ANSWER

from pyspark.sql.types import StructType, StructField, StringType
from pyspark.sql.functions import col

schema = StructType([
  StructField("SMS", StringType(), True)
])

In [37]:
# TEST - Run this cell to test your solution
fields = schema.fields

dbTest("ET1-P-05-02-01", 1, len(fields))
dbTest("ET1-P-05-02-02", 'SMS', fields[0].name)

print("Tests passed!")

Apply the schema to the data and save the result as `SMSDF`. This closes the loop on which to iterate and develop an increasingly complex schema. The path to the data is `/mnt/training/UbiqLog4UCI/14_F/log*`. 

Include only records where the column `SMS` is not `Null`.

In [39]:
# ANSWER
from pyspark.sql.types import StructType, StructField, StringType
from pyspark.sql.functions import col

schema = StructType([
  StructField("SMS", StringType(), True)
])

SMSDF = (spark.read
  .schema(schema)
  .json("/mnt/training/UbiqLog4UCI/14_F/log*")
  .filter(col("SMS").isNotNull())
)

display(SMSDF)

In [40]:
# TEST - Run this cell to test your solution
cols = SMSDF.columns

dbTest("ET1-P-05-03-01", 1147, SMSDF.count())
dbTest("ET1-P-05-03-02", ['SMS'], cols)

print("Tests passed!")

-sandbox
### Step 2: Create the Full Schema for SMS

Define the Schema for the following fields in the `StructType` `SMS` and name it `schema2`.  Apply it to a new DataFrame `SMSDF2`:  
<br>
* `Address`
* `date`
* `metadata`
 - `name`
 
<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> Note there's `Type` and `type`, which appears to be redundant data.

In [42]:
# ANSWER

from pyspark.sql.types import StructType, StructField, StringType
from pyspark.sql.functions import col

schema2 = StructType([
  StructField("SMS", StructType([
    StructField("Address",StringType(),True),
    StructField("date",StringType(),True),
    StructField("metadata", StructType([
      StructField("name",StringType(), True)
    ]), True),
  ]), True)
])

# Here is the full schema as well
# fullSchema = StructType([
#   StructField("SMS", StructType([
#     StructField("Address",StringType(),True),
#     StructField("Type",StringType(),True),
#     StructField("body",StringType(),True),
#     StructField("date",StringType(),True),
#     StructField("metadata", StructType([
#       StructField("name",StringType(), True)
#     ]), True),
#     StructField("type",StringType(),True)
#   ]), True)
# ])

SMSDF2 = (spark.read
  .schema(schema2)
  .json("/mnt/training/UbiqLog4UCI/14_F/log*")
  .filter(col("SMS").isNotNull()))

display(SMSDF2)

In [43]:
# TEST - Run this cell to test your solution
cols = SMSDF2.columns
schemaJson = SMSDF2.schema.json()

dbTest("ET1-P-05-04-01", 1147, SMSDF2.count())
dbTest("ET1-P-05-04-02", ['SMS'], cols)
dbTest("ET1-P-05-04-03", True, 'Address' in schemaJson and 'date' in schemaJson)

print("Tests passed!")

### Step 3: Compare Solution Performance

Compare the dafault schema inference to applying a user defined schema using the `%timeit` function.  Which completed faster?  Which triggered more jobs?  Why?

In [45]:
%timeit SMSDF = spark.read.schema(schema2).json("/mnt/training/UbiqLog4UCI/14_F/log*").count()

In [46]:
%timeit SMSDF = spark.read.json("/mnt/training/UbiqLog4UCI/14_F/log*").count()

Providing a schema increases performance two to three times, depending on the size of the cluster used. Since Spark doesn't infer the schema, it doesn't have to read through all of the data. This is also why there are fewer jobs when a schema is provided: Spark doesn't need one job for each partition of the data to infer the schema.

## Review

**Question:** What are two ways to attain a schema from data?  
**Answer:** Allow Spark to infer a schema from your data or provide a user defined schema. Schema inference is the recommended first step; however, you can customize this schema to your use case with a user defined schema.

**Question:** Why should you define your own schema?  
**Answer:** Benefits of user defined schemas include:
* Avoiding the extra scan of your data needed to infer the schema
* Providing alternative data types
* Parsing only the fields you need

**Question:** Why is JSON a common format in big data pipelines?  
**Answer:** Semi-structured data works well with hierarchical data and where schemas need to evolve over time.  It also easily contains composite data types such as arrays and maps.

**Question:** By default, how are corrupt records dealt with using `spark.read.json()`?  
**Answer:** They appear in a column called `_corrupt_record`.  These are the records that Spark can't read (e.g. when characters are missing from a JSON string).

## Next Steps

Start the next lesson, [Corrupt Record Handling]($./06-Corrupt-Record-Handling ).